# XML einlesen

In [ ]:
import xml.etree.ElementTree as ET

def read_xml_file(file_path):
  """Liest eine XML-Datei ein und gibt den Wurzelknoten zurück."""
  try:
    tree = ET.parse(file_path)
    root = tree.getroot()
    return root
  except FileNotFoundError:
    print(f"Datei nicht gefunden: {file_path}")
    return None
  except ET.ParseError:
    print(f"Fehler beim Parsen der XML-Datei: {file_path}")
    return None


In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
root = read_xml_file(fn)



In [ ]:
# prompt: in einem plot ausgeben

import numpy as np
import matplotlib.pyplot as plt

if root:
  yklb_values = []
  xklb_values = []
  for element in root.findall(".//MAKROKONTUR"):
    for child in element:
      if child.tag == 'KLB_DATEN':
        for child2 in child:
          if child2.tag == 'ROW':
            yklb_found = False
            xklb_found = False
            for child3 in child2:
              if child3.tag == 'YKLB':
                yklb_values.append(float(child3.text))
                yklb_found = True
              elif child3.tag == 'XKLB':
                xklb_values.append(float(child3.text))
                xklb_found = True
            if not yklb_found:
              yklb_values.append(np.nan)  # Füge NaN hinzu, falls YKLB fehlt
            if not xklb_found:
              xklb_values.append(np.nan)  # Füge NaN hinzu, falls XKLB fehlt

  yklb_array = np.array(yklb_values)
  xklb_array = np.array(xklb_values)

  # Plot erstellen
  plt.plot(xklb_array, yklb_array, marker='o', linestyle='-')
  plt.xlabel("XKLB")
  plt.ylabel("YKLB")
  plt.title("Plot von YKLB gegen XKLB")
  plt.grid(True)
  plt.show()
